# This workbook applies the obt diagonalizing rotation

In [1]:
import numpy as np
from dmrghandler.src.dmrghandler.qchem_dmrg_calc import single_qchem_dmrg_calc
from dmrghandler.src.dmrghandler.dmrg_looping import dmrg_central_loop
import dmrghandler.src.dmrghandler.dmrg_calc_prepare as dmrg_calc_prepare
from matplotlib import pyplot as plt
from scipy.optimize import curve_fit
from block2 import SZ as block2_SZ
from get_dmrg_config import get_dmrg_process_param, get_dmrg_config
from pathlib import Path
from pyblock2.driver.core import DMRGDriver, SymmetryTypes
from pyblock2 import tools
from pyblock2._pyscf.ao2mo import integrals as itg
import pyscf
from eigenstate_check import check_eigenstates
from basic_definitions import chemist_to_physicist
from Hamiltonians.molecular_hamiltonians import get_n2, get_h2o

In [2]:
def h1e_hubbard(t, ncas):
    tensor = np.zeros((ncas, ncas))
    for i in range(ncas - 1):
        tensor[i, i + 1] = -t
        tensor[i + 1, i] = -t
    
    return tensor

def g2e_hubbard(U, ncas):
    tensor = np.zeros((ncas, ncas, ncas, ncas))
    for i in range(ncas):
        tensor[i, i, i, i] = U
        
    return tensor


ncas = 4
n_elec = 4
spin = 0

h1e = h1e_hubbard(1, ncas)
g2e = g2e_hubbard(1, ncas)

data = get_h2o(bond_length=3.0)

ncas, n_elec, spin, h1e, g2e = data.ncas, data.n_elec, data.spin, data.h1e, data.g2e

converged SCF energy = -74.271032118415


In [5]:
from pyscf import gto, scf

mol = gto.M(
    atom=f"N 0 0 0; N 0 0 2.2",
    basis='sto-3g')
mf = scf.RHF(mol).run(conv_tol=1E-14)
mycc = mf.CISD().run()
ncas, n_elec, spin, ecore, h1e, g2e, orb_sym = itg.get_rhf_integrals(mf,
                                                                     ncore=0,
                                                                     g2e_symm=1)


label = "_obt_diag"
filename = "fcidump.N2_22" + f"{label}"
pyscf.tools.fcidump.from_integrals(
    "fcidump_output"/Path(filename),
    h1e,
    g2e,
    nmo=h1e.shape[0],
    nelec=n_elec,
    nuc=ecore,
    ms=spin,
    orbsym=None,
    tol=1E-8,
    float_format=' %.16g',
)

converged SCF energy = -106.751831266188
E(RCISD) = -107.2227326317978  E_corr = -0.4709013656102005


In [6]:
num_orbitals = ncas
num_electrons = n_elec
num_unpaired = spin
multiplicity = 2 * spin + 1

dmrg_params = get_dmrg_config(
    num_orbitals, num_electrons, num_unpaired,
    multiplicity)

absorbed_tbt = pyscf.fci.direct_nosym.absorb_h1e(np.array(h1e), np.array(g2e), norb=num_orbitals, nelec=num_electrons)
h_in_tbt = pyscf.ao2mo.restore(1, absorbed_tbt.copy(), num_orbitals).astype(np.array(h1e).dtype, copy=False)
one_body_dummy = np.zeros((num_orbitals, num_orbitals))
obt, tbt = chemist_to_physicist(one_body_dummy, h_in_tbt)

In [88]:
loop_results = dmrg_central_loop(obt, tbt, dmrg_params, 1000, 1e6, 1e-300   , "./mps_directory", verbosity=1, move_mps_to_final_storage_path=None)

integral cutoff error =  0.0
mpo terms =       4116

Build MPO | Nsites =     7 | Nterms =       4116 | Algorithm = FastBIP | Cutoff = 1.00e-120
 Site =     0 /     7 .. Mmpo =    26 DW = 0.00e+00 NNZ =       26 SPT = 0.0000 Tmvc = 0.000 T = 0.001
 Site =     1 /     7 .. Mmpo =    70 DW = 0.00e+00 NNZ =      287 SPT = 0.8423 Tmvc = 0.000 T = 0.002
 Site =     2 /     7 .. Mmpo =   114 DW = 0.00e+00 NNZ =      567 SPT = 0.9289 Tmvc = 0.000 T = 0.002
 Site =     3 /     7 .. Mmpo =   114 DW = 0.00e+00 NNZ =     2105 SPT = 0.8380 Tmvc = 0.000 T = 0.002
 Site =     4 /     7 .. Mmpo =    70 DW = 0.00e+00 NNZ =      567 SPT = 0.9289 Tmvc = 0.000 T = 0.001
 Site =     5 /     7 .. Mmpo =    26 DW = 0.00e+00 NNZ =      287 SPT = 0.8423 Tmvc = 0.000 T = 0.001
 Site =     6 /     7 .. Mmpo =     1 DW = 0.00e+00 NNZ =       26 SPT = 0.0000 Tmvc = 0.000 T = 0.000
Ttotal =      0.009 Tmvc-total = 0.001 MPO bond dimension =   114 MaxDW = 0.00e+00
NNZ =         3865 SIZE =        32648 SPT = 0.8816

/Users/rick/PycharmProjects/croor/dmrghandler/src/dmrghandler/energy_extrapolation.py:345: RuntimeWarning: divide by zero encountered in log
  ln_del_E_rel = a + b * np.log(discarded_weights)


In [89]:
loop_results["finish_reason"]

'Energy change below threshold, limit 1e-300, achieved 0.0'

In [90]:
symmetry_type = SymmetryTypes.SZ
stack_mem = 10*1024*1024*1024
stack_mem_ratio = 0.5
num_threads = 3
restart_dir=Path("./temp")
mps_dir = Path("./mps_directory/mps_storage/dmrg_loop_030_ket_optimized")
driver = DMRGDriver(
    stack_mem=stack_mem,
    scratch=str(mps_dir),
    clean_scratch=True,  # Default value
    restart_dir=str(restart_dir),
    n_threads=num_threads,
    # n_mkl_threads=n_mkl_threads,  # Default value is 1
    symm_type=symmetry_type,
    mpi=None,  # Default value
    stack_mem_ratio=stack_mem_ratio,  # Default value 0.4
    fp_codec_cutoff=1e-120,  # Default value 1e-16,
)
tools.init(block2_SZ)
driver.initialize_system(n_sites=num_orbitals, n_elec=num_electrons, spin=num_unpaired, orb_sym=None)
mps = tools.loadMPSfromDir(mps_info=None, mpsSaveDir=mps_dir)
csfs, coeffs = driver.get_csf_coefficients(mps, cutoff=1e-308, iprint=1)
variance = check_eigenstates(num_orbitals, num_electrons, num_unpaired, obt, tbt, "dmrg_loop_030_ket_optimized", "mps_directory")
print(f"Energy variance: {variance}")

RuntimeError: MPSInfo:load_data on 'mps_directory/mps_storage/dmrg_loop_030_ket_optimized/mps_info.bin' failed.

In [7]:
import scipy
eig, eigv = scipy.linalg.eigh(h1e)

# Verify eigenvectors being unitary
product = eigv.conj().T @ eigv
print(np.allclose(product, np.eye(eigv.shape[0])))

True


In [8]:
U_NO = eigv
print(U_NO)

[[ 9.99627385e-01  6.80520894e-07  2.28386479e-02  1.49246202e-08
   1.58541344e-16  2.28630317e-17  1.99164753e-16 -1.52975010e-16
  -1.02286873e-08 -1.49494959e-02]
 [-6.80507604e-07  9.99646878e-01 -1.55512255e-08  2.19191362e-02
   1.17373756e-16  7.47365882e-17  1.75134657e-16 -3.33316050e-17
  -1.50223312e-02  1.01793290e-08]
 [-1.87054373e-11 -2.62884523e-02  4.33113344e-10  8.98245488e-01
   4.32371304e-16  4.30501996e-15  7.90773238e-15  2.58308619e-15
  -4.38707147e-01 -3.52977776e-10]
 [ 2.71172880e-02 -1.87677018e-11 -7.68263094e-01  5.16187435e-10
  -7.10107012e-15 -3.56980551e-15 -1.01894280e-14  5.80251394e-15
  -2.16194447e-10  6.39559591e-01]
 [ 6.50820949e-13 -3.87765947e-03 -9.65347624e-11 -4.38947144e-01
   1.28082673e-13  1.55370480e-15  1.49775612e-14 -6.90040423e-15
  -8.98504518e-01 -6.55563609e-11]
 [ 2.66832233e-16 -5.84537741e-17 -5.68600711e-14  5.18486383e-15
   9.65038506e-11 -9.89012486e-11  1.65371035e-01 -9.86231424e-01
   7.58471573e-15 -5.67400483e-14

## 3, Combine the obt into the tbt

In [9]:
U_NO = np.array(U_NO).conj().T

In [10]:
absorbed_tbt = pyscf.fci.direct_nosym.absorb_h1e(np.array(h1e), np.array(g2e), norb=num_orbitals, nelec=num_electrons)
h_in_tbt = pyscf.ao2mo.restore(1, absorbed_tbt.copy(), num_orbitals).astype(np.array(h1e).dtype, copy=False)

In [11]:
p = np.einsum_path('ak,bl,cm,dn,klmn->abcd', U_NO, U_NO, U_NO, U_NO, h_in_tbt)[0]
orbital_rotated = np.einsum('ak,bl,cm,dn,klmn->abcd', U_NO, U_NO, U_NO, U_NO, h_in_tbt, optimize = p)

In [12]:
num_orbitals = ncas
num_electrons = n_elec
num_unpaired = spin
multiplicity = 2 * spin + 1

dmrg_params = get_dmrg_config(
    num_orbitals, num_electrons, num_unpaired,
    multiplicity)
one_body_dummy = np.zeros((ncas, ncas))
obt, tbt = chemist_to_physicist(one_body_dummy, orbital_rotated)
label = "_obt_diag"
filename = "fcidump.N2_22" + f"{label}"
pyscf.tools.fcidump.from_integrals(
    "fcidump_output"/Path(filename),
    obt,
    tbt,
    nmo=obt.shape[0],
    nelec=n_elec,
    nuc=ecore,
    ms=spin,
    orbsym=None,
    tol=1E-8,
    float_format=' %.16g',
)

In [97]:
loop_results = dmrg_central_loop(obt, tbt, dmrg_params, 1000, 1e6, 1e-300   , "./mps_directory_2", verbosity=1, move_mps_to_final_storage_path=None)


integral cutoff error =  0.0
mpo terms =       4580

Build MPO | Nsites =     7 | Nterms =       4580 | Algorithm = FastBIP | Cutoff = 1.00e-120
 Site =     0 /     7 .. Mmpo =    30 DW = 0.00e+00 NNZ =       30 SPT = 0.0000 Tmvc = 0.001 T = 0.006
 Site =     1 /     7 .. Mmpo =    70 DW = 0.00e+00 NNZ =      325 SPT = 0.8452 Tmvc = 0.000 T = 0.002
 Site =     2 /     7 .. Mmpo =   114 DW = 0.00e+00 NNZ =      593 SPT = 0.9257 Tmvc = 0.000 T = 0.002
 Site =     3 /     7 .. Mmpo =   114 DW = 0.00e+00 NNZ =     2225 SPT = 0.8288 Tmvc = 0.000 T = 0.003
 Site =     4 /     7 .. Mmpo =    70 DW = 0.00e+00 NNZ =      583 SPT = 0.9269 Tmvc = 0.000 T = 0.002
 Site =     5 /     7 .. Mmpo =    30 DW = 0.00e+00 NNZ =      323 SPT = 0.8462 Tmvc = 0.000 T = 0.001
 Site =     6 /     7 .. Mmpo =     1 DW = 0.00e+00 NNZ =       30 SPT = 0.0000 Tmvc = 0.000 T = 0.001
Ttotal =      0.016 Tmvc-total = 0.002 MPO bond dimension =   114 MaxDW = 0.00e+00
NNZ =         4109 SIZE =        33216 SPT = 0.8763

/Users/rick/PycharmProjects/croor/dmrghandler/src/dmrghandler/energy_extrapolation.py:344: RuntimeWarning: divide by zero encountered in log
  a = np.log(alpha)  # α = exp(a), np.log is the natural logarithm


integral cutoff error =  0.0
mpo terms =       4580

Build MPO | Nsites =     7 | Nterms =       4580 | Algorithm = FastBIP | Cutoff = 1.00e-120
 Site =     0 /     7 .. Mmpo =    30 DW = 0.00e+00 NNZ =       30 SPT = 0.0000 Tmvc = 0.000 T = 0.001
 Site =     1 /     7 .. Mmpo =    70 DW = 0.00e+00 NNZ =      325 SPT = 0.8452 Tmvc = 0.000 T = 0.002
 Site =     2 /     7 .. Mmpo =   114 DW = 0.00e+00 NNZ =      593 SPT = 0.9257 Tmvc = 0.000 T = 0.002
 Site =     3 /     7 .. Mmpo =   114 DW = 0.00e+00 NNZ =     2225 SPT = 0.8288 Tmvc = 0.000 T = 0.003
 Site =     4 /     7 .. Mmpo =    70 DW = 0.00e+00 NNZ =      583 SPT = 0.9269 Tmvc = 0.000 T = 0.001
 Site =     5 /     7 .. Mmpo =    30 DW = 0.00e+00 NNZ =      323 SPT = 0.8462 Tmvc = 0.000 T = 0.001
 Site =     6 /     7 .. Mmpo =     1 DW = 0.00e+00 NNZ =       30 SPT = 0.0000 Tmvc = 0.000 T = 0.001
Ttotal =      0.010 Tmvc-total = 0.001 MPO bond dimension =   114 MaxDW = 0.00e+00
NNZ =         4109 SIZE =        33216 SPT = 0.8763

/Users/rick/PycharmProjects/croor/dmrghandler/src/dmrghandler/energy_extrapolation.py:344: RuntimeWarning: divide by zero encountered in log
  a = np.log(alpha)  # α = exp(a), np.log is the natural logarithm


integral cutoff error =  0.0
mpo terms =       4580

Build MPO | Nsites =     7 | Nterms =       4580 | Algorithm = FastBIP | Cutoff = 1.00e-120
 Site =     0 /     7 .. Mmpo =    30 DW = 0.00e+00 NNZ =       30 SPT = 0.0000 Tmvc = 0.000 T = 0.001
 Site =     1 /     7 .. Mmpo =    70 DW = 0.00e+00 NNZ =      325 SPT = 0.8452 Tmvc = 0.000 T = 0.002
 Site =     2 /     7 .. Mmpo =   114 DW = 0.00e+00 NNZ =      593 SPT = 0.9257 Tmvc = 0.000 T = 0.002
 Site =     3 /     7 .. Mmpo =   114 DW = 0.00e+00 NNZ =     2225 SPT = 0.8288 Tmvc = 0.000 T = 0.003
 Site =     4 /     7 .. Mmpo =    70 DW = 0.00e+00 NNZ =      583 SPT = 0.9269 Tmvc = 0.000 T = 0.001
 Site =     5 /     7 .. Mmpo =    30 DW = 0.00e+00 NNZ =      323 SPT = 0.8462 Tmvc = 0.000 T = 0.001
 Site =     6 /     7 .. Mmpo =     1 DW = 0.00e+00 NNZ =       30 SPT = 0.0000 Tmvc = 0.000 T = 0.000
Ttotal =      0.010 Tmvc-total = 0.001 MPO bond dimension =   114 MaxDW = 0.00e+00
NNZ =         4109 SIZE =        33216 SPT = 0.8763

...

/Users/rick/PycharmProjects/croor/dmrghandler/src/dmrghandler/energy_extrapolation.py:344: RuntimeWarning: divide by zero encountered in log
  a = np.log(alpha)  # α = exp(a), np.log is the natural logarithm


integral cutoff error =  0.0
mpo terms =       4580

Build MPO | Nsites =     7 | Nterms =       4580 | Algorithm = FastBIP | Cutoff = 1.00e-120
 Site =     0 /     7 .. Mmpo =    30 DW = 0.00e+00 NNZ =       30 SPT = 0.0000 Tmvc = 0.000 T = 0.001
 Site =     1 /     7 .. Mmpo =    70 DW = 0.00e+00 NNZ =      325 SPT = 0.8452 Tmvc = 0.000 T = 0.002
 Site =     2 /     7 .. Mmpo =   114 DW = 0.00e+00 NNZ =      593 SPT = 0.9257 Tmvc = 0.000 T = 0.002
 Site =     3 /     7 .. Mmpo =   114 DW = 0.00e+00 NNZ =     2225 SPT = 0.8288 Tmvc = 0.000 T = 0.003
 Site =     4 /     7 .. Mmpo =    70 DW = 0.00e+00 NNZ =      583 SPT = 0.9269 Tmvc = 0.000 T = 0.001
 Site =     5 /     7 .. Mmpo =    30 DW = 0.00e+00 NNZ =      323 SPT = 0.8462 Tmvc = 0.000 T = 0.001
 Site =     6 /     7 .. Mmpo =     1 DW = 0.00e+00 NNZ =       30 SPT = 0.0000 Tmvc = 0.000 T = 0.000
Ttotal =      0.010 Tmvc-total = 0.001 MPO bond dimension =   114 MaxDW = 0.00e+00
NNZ =         4109 SIZE =        33216 SPT = 0.8763

/Users/rick/PycharmProjects/croor/dmrghandler/src/dmrghandler/energy_extrapolation.py:344: RuntimeWarning: divide by zero encountered in log
  a = np.log(alpha)  # α = exp(a), np.log is the natural logarithm


integral cutoff error =  0.0
mpo terms =       4580

Build MPO | Nsites =     7 | Nterms =       4580 | Algorithm = FastBIP | Cutoff = 1.00e-120
 Site =     0 /     7 .. Mmpo =    30 DW = 0.00e+00 NNZ =       30 SPT = 0.0000 Tmvc = 0.000 T = 0.002
 Site =     1 /     7 .. Mmpo =    70 DW = 0.00e+00 NNZ =      325 SPT = 0.8452 Tmvc = 0.000 T = 0.002
 Site =     2 /     7 .. Mmpo =   114 DW = 0.00e+00 NNZ =      593 SPT = 0.9257 Tmvc = 0.000 T = 0.002
 Site =     3 /     7 .. Mmpo =   114 DW = 0.00e+00 NNZ =     2225 SPT = 0.8288 Tmvc = 0.000 T = 0.003
 Site =     4 /     7 .. Mmpo =    70 DW = 0.00e+00 NNZ =      583 SPT = 0.9269 Tmvc = 0.000 T = 0.001
 Site =     5 /     7 .. Mmpo =    30 DW = 0.00e+00 NNZ =      323 SPT = 0.8462 Tmvc = 0.000 T = 0.001
 Site =     6 /     7 .. Mmpo =     1 DW = 0.00e+00 NNZ =       30 SPT = 0.0000 Tmvc = 0.000 T = 0.000
Ttotal =      0.010 Tmvc-total = 0.001 MPO bond dimension =   114 MaxDW = 0.00e+00
NNZ =         4109 SIZE =        33216 SPT = 0.8763

/Users/rick/PycharmProjects/croor/dmrghandler/src/dmrghandler/energy_extrapolation.py:344: RuntimeWarning: divide by zero encountered in log
  a = np.log(alpha)  # α = exp(a), np.log is the natural logarithm
/Users/rick/PycharmProjects/croor/dmrghandler/src/dmrghandler/energy_extrapolation.py:345: RuntimeWarning: divide by zero encountered in log
  ln_del_E_rel = a + b * np.log(discarded_weights)


In [76]:
loop_results["finish_reason"]

'Energy change below threshold, limit 1e-300, achieved 0.0'

In [77]:
symmetry_type = SymmetryTypes.SZ
stack_mem = 10*1024*1024*1024
stack_mem_ratio = 0.5
num_threads = 3
restart_dir=Path("./temp")
mps_dir = Path("./mps_directory_2/mps_storage/dmrg_loop_026_ket_optimized")
driver = DMRGDriver(
    stack_mem=stack_mem,
    scratch=str(mps_dir),
    clean_scratch=True,  # Default value
    restart_dir=str(restart_dir),
    n_threads=num_threads,
    # n_mkl_threads=n_mkl_threads,  # Default value is 1
    symm_type=symmetry_type,
    mpi=None,  # Default value
    stack_mem_ratio=stack_mem_ratio,  # Default value 0.4
    fp_codec_cutoff=1e-120,  # Default value 1e-16,
)
tools.init(block2_SZ)
driver.initialize_system(n_sites=num_orbitals, n_elec=num_electrons, spin=num_unpaired, orb_sym=None)
mps = tools.loadMPSfromDir(mps_info=None, mpsSaveDir=mps_dir)
csfs, coeffs = driver.get_csf_coefficients(mps, cutoff=1e-308, iprint=1)
variance = check_eigenstates(num_orbitals, num_electrons, num_unpaired, obt, tbt, "dmrg_loop_026_ket_optimized", "mps_directory_2")
print(f"Energy variance: {variance}")

RuntimeError: MPSInfo:load_data on 'mps_directory_2/mps_storage/dmrg_loop_026_ket_optimized/mps_info.bin' failed.